# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
#общая информация по данным
df.info()
#расчет количества уникальных значений в столбце с количеством детей и среднее количество
df['children'].value_counts()
df['children'].mean()
#расчет количества уникальных значений возраста
df['dob_years'].value_counts()
#приведение всех строк столбца с образованием к нижнему регистру
df['education'] = df['education'].str.lower()
#определение категорий из столбца 'education_id', соответствующих уровню образования из столбца 'education'
df[df['education_id'] == 4]['education'].unique()
#определение уникальных значений столбца с семейным положением, приведение строк к нижнему регистру
df['family_status'].unique()
df['family_status'] = df['family_status'].str.lower()
#определение количества уникальных значений столбца с полом
df['gender'].value_counts()
#определение соответствия категорий семейному положению
df[df['family_status_id'] == 0]['family_status'].unique()
#определение количества уникальных значений типа занятости
df['income_type'].value_counts()
#определение количества уникальных значений столбца задолженности по кредиту
df['debt'].value_counts()
#определение количества пропущенных значений в столбце с заработной платой и попытка анализа, связаны ли пропуски со значениями в дугих столбцах
df[df['total_income'].isna()].head(10)
#определение количества уникальных значений целей кредита
df['purpose'].value_counts()
df.describe([.2, .4, .6, .8])


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
20%,0.000000,-3212.156113,31.000000,0.000000,0.000000,0.000000,9.483683e+04
40%,0.000000,-1687.668878,39.000000,1.000000,0.000000,0.000000,1.282797e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
60%,0.000000,-819.649418,47.000000,1.000000,1.000000,0.000000,1.646609e+05
80%,1.000000,-150.955390,56.000000,1.000000,2.000000,0.000000,2.231076e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Вывод**

При обработке данных обнаружены следующие проблемы:
1. В столбцах 'days_employed' и 'total_income' пропущены значения.
2. В столбце 'children' встречаются значения [ 1,  0,  3,  2, -1,  4, 20,  5]. Очевидно, что отрицательное значение в столбце количества детей является ошибкой внесения данных. При этом при расчете значений методом value_counts() для значения 20 получено 76 результатов, что противоречит логике, поэтому также является ошибкой внесения данных. Далее данные значения будут исправлены в предположении, что 20- это неверно записанная 2, так как в NumPad 0 стоит под цифрой 2 и может быть случайно нажат при печати, а -1 будет исправлен на среднее значение, округленное до целого- 1.
3. В столбце 'days_employed' при анализе первых и последних 10 строк таблицы методами head() и tail(), соответственно, а также методами min(), max(), mean() и median() выявлено, что в столбце встречаются значения от -18388 до 401755, что соответствует -50 и 1100 годам трудового стажа. Также в данном столбце методом isna() и sum() выявлено 2174 пропущенных значения. Информация, содержащаяся в данном столбце, не будет использована при исследовании, поэтому для дальнейшей работы с таблицей достаточно заменить отрицательные значения в нем на положительные, а пропуски- заполнить медианным значением. Также необходимо изменить вещественный тип данных в столбце на целочисленный, так как информация в нем должна быть указана в полных днях.
4. В столбце 'dob_years' выявлено 101 нулевое значение. Остальные строки принимают значения от 19 до 75. Так как нулевые значения не оказывают сатистически значимого влияния (менее 0,5%), они могут быть заменены медианным значением.
5. В столбце 'education' при вызове метода unique() обнаружены посторяющиеся значения, записанные верхним и нижним регистром, которые приведены к нижнему регистру методом str.lower(). Категории в столбце 'education_id' распределены следующим образом: 0- высшее образование, 1- среднее образование, 2- неоконченное высшее, 3- начальное, 4- ученая степень. Фактически столбцы 'education' и 'education_id' содержат в себе идентичную информацию и в дальнейшем может использоваться только один из них.
6. Столбцы 'family_status' и 'family_status_id' также содержат идентичную информацию. Данные в столбце 'family_status' приведены к нижнему регистру. Категории в столбце 'family_status_id' распределены следующим образом: 0- женат / замужем, 1- гражданский брак, 2- вдовец / вдова, 3- в разводе, 4- не женат / не замужем.
7. В столбце 'gender' найдено три типа значений- F, M и XNA. Так как значение XNA встречается только один раз и не оказывает статистически значимого влияния, то оно может быть заменено любым из двух оставшихся значений.
8. В столбце 'income_type' выявлены следующие значения- 'сотрудник', 'пенсионер', 'компаньон', 'госслужащий', 'безработный', 'предприниматель', 'студент', 'в декрете'.
9. В столбце 'debt' нет пропусков данных и она содержит только два типа значений- 0 и 1. В описании данных столбец 'debt' подписан как 'имел ли задолженность по возврату кредитов'. Отсюда можно сделать вывод, что 0 означает, что клиент не имел задолженностей по возврату, а 1- имел задолженность.
10. В столбце 'total_income' содержатся значения от 20667 до 2265604, среднее- 167422, медианное- 145017. Одной из гипотез в исследовании является зависимость частоты возникновения задолженности по кредиту от размера заработной платы, поэтому информация в данном столбце является важной для целей исследования. В столбце пропущено 2174 значения- это около 10% от всего количества строк в таблице. В дальнейшем  данные пропуски будут заполнены медианным значением зарплаты в выборке по полу, типу занятости и уровню образования. Данная выборка основана на преположениях о том, что заработная плата мужчин статистически выше, чем у женщин (https://trends.rbc.ru/trends/social/60816b4c9a7947a3a27e8e87), уровень образования напрямую влияет на размер заработной платы- чем выше уровень, тем выше зарплата, а тип занятости определяет вилку зарплат. Важно отметить, что пропуски в столбцах 'days_employed' и 'total_income' встречаются в одних и тех же строках. Необходимо определить источник данных и проанализировать источник ошибки при их переносе. Также необходимо изменить вещественный тип данных в столбце на целочисленный.
11. В столбце 'purpose' описаны цели кредита. При анализе данных, содержащихся в столбце выявлено большое количество различных целей, однако зачастую это одна цель, описанная разными словами. Например- 'сыграть свадьбу' и 'на проведение свадьбы'. Информация в данном столбце в дальнейшем потребует лемматизации и приведения к единой форме каждой цели кредита: 'проведение свадьбы', 'покупка недвижимости', 'строительство и ремонт', 'приобретение автомобиля', 'образование'. Следует рассмотреть возможность внесения данных в данный столбец в виде выпадающего списка, а не текстовой строки. Это позволит уточнить сбор данных и избежать ситуаций, когда сотрудник записывает цель кредита 'покупка недвижимости' без уточнения- жилая эта недвижимость или коммерческая. Такая ситуация не позволяет категоризировать данные по виду недвижимости и, соответственно, проверить гипотезу о том, что вид недвижимости влияет на задолженности по кредиту.
12. Получена информация о разбросе значений в каждом столбце методом describe(). Получены минимальное и максимальное значение, а также 20, 4, 60 и 80 процентили по столбцу дохода, которые будут учтены далее при категоризации данных.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
#исправление отрицательных значений в столбце стажа, заполнение пропусков медианным значением по возрасту и проверка пропусков в столбце
df['days_employed'] = abs(df['days_employed'])
df_de_trans = df.groupby('dob_years')['days_employed'].transform('median')
df['days_employed'] = df['days_employed'].fillna(df_de_trans)
df['days_employed'].isna().sum()

0

In [3]:
#изменение значения 'XNA' в столбце 'gender'
#df.loc[df['gender'] == 'XNA', 'gender'] = 'F'
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [4]:
#заполнение и проверка пропусков в столбце 'total_income' в выборке по полу, типу занятости и уровню образования
df_trans = df.groupby(['gender', 'income_type', 'education_id'])['total_income'].transform('median')
df['total_income'] = df['total_income'].fillna(df_trans)
df.groupby(['gender', 'income_type', 'education_id'])['total_income'].median()
#обнаружен пропуск в данных для мужчин-предпринимателей со средним образованием.
#Так как предпринимателей с другим уровнем образования в данных нет, потребуется заполнение пропуска без учета пола и образования
df_trans_1 = df.groupby('income_type')['total_income'].transform('median')
df['total_income'] = df['total_income'].fillna(df_trans_1)
df.groupby(['gender', 'income_type', 'education_id'])['total_income'].min()

gender  income_type      education_id
F       безработный      0               202722.511368
        в декрете        1                53829.130729
        госслужащий      0                44734.262614
                         1                29200.077193
                         2                50962.678837
                         3                73471.521358
                         4               111392.231107
        компаньон        0                45654.214129
                         1                28702.812889
                         2                59702.378573
                         3                71051.529875
        пенсионер        0                35137.992145
                         1                20667.263793
                         2                36445.340832
                         3                25308.586849
                         4               255425.196556
        предприниматель  0               499163.144947
        сотрудник        0 

In [5]:
#замена отрицательных значений в столбце с количеством детей на положительные
df['children'] = abs(df['children'])

#замена значения 20 в столбце с количеством детей на 2, исходя из предположения, 
#что эта ошибка внесения данных вызвана расположением клавиш в панели NumPad
df.loc[df['children'] == 20, 'children'] = 2
df['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

**Вывод**

1. На данном этапе были изменены некорректные значения в столбце 'gender'. Так как некорректное значение 'XNA' было только в одном столбце, оно было заменено в произвольном порядке на значение 'F'.
2. В столбце 'days_employed' отрицательные значения были исправлены на положительные, а пропуски заполнены медианным значением. Принимая во внимание качество данных в данном столбце- разброс значений от отрицательных до 1100 лет трудового стажа, формулирование гипотез далее будет производиться без учета трудового стажа.
3. В столбце 'total_income' пропуски были заполнены медианным значением в выборке по полу, виду деятельности и уровню образования. Однако для мужчин-предпринимателей со средним образованием такое заполнение не дало результата. Так как предприниматели в датафрейме имеют только средний уровень образования, пропуски были заполнены исходя только из вида деятельности без учета образования и пола.
4. В столбце 'children' данные исправлены, исходя из предположения, что 20- это неверно записанная 2, так как в NumPad 0 стоит под цифрой 2 и может быть случайно нажат при печати, а значение -1 исправлено на среднее значение, округленное до целого- 1.

### Замена типа данных

In [6]:
#замена типа данных в столбце 'days_employed' на целочисленный
df['days_employed'] = df['days_employed'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


In [7]:
#замена типа данных в столбце 'total_income' на целочисленный
df['total_income'] = df['total_income'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

В столбцах 'days_employed' и 'total_income', соответствующих количеству отработанных дней и ежемесячному доходу, вещественный тип данных был заменен на целочисленный.

### Обработка дубликатов

In [8]:
#расчет количества явных дубликатов
df.duplicated().sum()
#найден 71 явный дубликат. Удаление явных дубликатов с обновлением индексов:
df = df.drop_duplicates().reset_index(drop=True)
df.duplicated().sum()

0

**Вывод**

Поиском был обнаружен 71 явный дубликат. Дубликаты были удалены с обновлением индексов в таблице.

### Лемматизация

In [9]:
#лемматизация столбца 'purpose' для приведеня целей кредита к единой форме
from pymystem3 import Mystem
m = Mystem()
def purpose_lemmatizing(row):
    purpose = str(row['purpose'])
    if 'автомобиль' in m.lemmatize(purpose):
        return 'покупка автомобиля'
    if 'свадьба' in m.lemmatize(purpose):
        return 'проведение свадьбы'
    if ('строительство' in m.lemmatize(purpose)) or ('ремонт' in m.lemmatize(purpose)):
        return 'строительство и ремонт'
    if ('недвижимость' in m.lemmatize(purpose)) or ('жилье' in m.lemmatize(purpose)):
        return 'покупка недвижимости'
    if 'образование' in m.lemmatize(purpose):
        return 'образование'
    else:
        return purpose
df['purpose'] = df.apply(purpose_lemmatizing, axis=1)
df['purpose'].value_counts()


покупка недвижимости      8326
покупка автомобиля        4306
образование               4013
строительство и ремонт    2485
проведение свадьбы        2324
Name: purpose, dtype: int64

**Вывод**

В результате лемматизации данных в столбце с целями кредита было выделено 5 целей кредита- покупка недвижимости, покупка автомобиля, образование, строительство и ремонт, проведение свадьбы. Строительство и ремонт недвижимости отделены от покупки недвижимости в связи с тем, что строительство и ремонт более растянуты по времени, чем покупка, и риски заемщика повышаются, так как на успешное завершение строительства и ремонта без привлечения дополнительных средств могут действовать большое количество факторов. Так как информация в данный столбец заносится через тесктовую строку, не во всех случаях был выделен вид недвижимости- коммерческая или жилая, вид автомобиля- новый или подержанный, вид образования- высшее или дополнительное. Данную проблему можно решить изменением типа внесения данных на выпадающий список. В таком случае категоризация данных по данному столбцу может быть уточнена и могут быть сформулированы дополнительные гипотезы для проверки.

### Категоризация данных

В данном разделе будут сформулированы гипотезы для проверки, а также будет произведена категоризация данных для проверки гипотез.
Гипотезы для проверки следующие:
#### Влияет ли количество детей на возврат кредита в срок?

Данная гипотеза потребует категоризации данных по наличию детей у заемщика. Здесь предполагается, что наличие детей у заемщика заставляет его подходить ответственнее к выплатам по кредиту и ожидается, что возврат кредита в срок при наличии детей происходит чаще. Для проверки гипотезы необходимо ввести столбец 'is_children' с категориями 0- 'нет детей' и 1- 'есть дети'.

In [10]:
#проверка первой гипотезы. Ввод доболнительного столбца 'is_children' 
def is_children(children):
    if children == 0:
        return 0
    else:
        return 1
df['is_children'] = df['children'].apply(is_children)
df['is_children'].value_counts()

0    14091
1     7363
Name: is_children, dtype: int64

#### Влияет ли семейное положение на возврат кредита в срок?

Данная гипотеза потребует вычисления процента возвратов кредита в срок в выборке по семейному положению заемщика. Здесь предполагается, что наличие у заемщика партнера заставляет его подходить ответственнее к выплатам по кредиту и ожидается, что возврат кредита в срок при наличии партнера происходит чаще.

In [11]:
#проверка второй гипотезы. Создание сводной таблицы с расчетом процента задержек в категории по семейному положению
df['family_status'].value_counts()

женат / замужем          12339
гражданский брак          4151
не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

#### Влияет ли уровень дохода на возврат кредита в срок?
Данная гипотеза потребует категоризации по уровню доходов. Для проверки гипотезы данные в столбце с ежемесячным доходом будут поделены на 5 категорий- 20-й, 40-й, 60-й, 80-й и 100-й процентили.

In [12]:
#создание дополнительного столбца с выделением категории по уровню дохода
def total_income_type(total_income):
    if total_income < df.total_income.quantile(.2):
        return '20-й процентиль'
    elif total_income < df.total_income.quantile(.4):
        return '40-й процентиль'
    elif total_income < df.total_income.quantile(.6):
        return '60-й процентиль'
    elif total_income < df.total_income.quantile(.8):
        return '80-й процентиль'
    else:
        return '100-й процентиль'
df['total_income_type'] = df['total_income'].apply(total_income_type)
df['total_income_type'].value_counts()


100-й процентиль    4291
20-й процентиль     4291
80-й процентиль     4291
40-й процентиль     4291
60-й процентиль     4290
Name: total_income_type, dtype: int64

#### Влияет ли цель кредита навозврат кредита в срок?
Данная гипотеза потребует вычисления доли возвратов кредита в срок в зависимости от цели кредита.

In [13]:
df['purpose'].value_counts()

покупка недвижимости      8326
покупка автомобиля        4306
образование               4013
строительство и ремонт    2485
проведение свадьбы        2324
Name: purpose, dtype: int64

#### Влияет ли тип занятости на возврат кредита в срок?
Данная гипотеза потребует расчета доли  возврата кредита в срок в зависимости от типа занятости заемщика. Предполагается, что наличие постоянной работы у заемщика позволяет лучше планировать и выплачивать кредит в срок.

In [14]:
df['income_type'].value_counts()

сотрудник          11084
компаньон           5078
пенсионер           3829
госслужащий         1457
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

#### Влияет ли уровень образования на возвращение кредита в срок?
Данная гипотеза потребует вычисления доли возвратов кредита в срок в зависимости от уровня образования. Предполагается, что наличие образования у заемщика выше среднего повышает его финансовую гармотность и приводит к меньшему числу случаев задержек по кредитам.

In [15]:
df['education'].value_counts()

среднее                15172
высшее                  5250
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

**Вывод**

В разделе 2.5 проекта выполнена категоризация данных для проверки гипотез по следующим категориям:
* по наличию у заемщика детей;
* по семейному положению;
* по уровню дохода;
* по цели кредита;
* по типу занятости;
* по уровню образования.

В первых четырех категориях выборки достаточны для проведения исследования и проверки гипотезы. В выборке по типу занятости для категорий 'безработный', 'студент', 'в декрете' и 'предприниматель' количества данных недостаточно для получения статистически достоверного результата, но по остальным категориям гипотеза может быть проверена. В выборке по уровню образования для категории 'ученая степени' количества данных также недостаточно для получения статистически достоверного результата, а к резултатам по категориям 'неоконченное высшее' и 'начальное' необходимо подходить с осторожностью, так как данных по ним намного меньше, чем по остальным категориям.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [16]:
#сравнение задержек по кредитам в категориях с детьми и без детей со средним значением и печать результата
avg_debt = df['debt'].sum() / df['debt'].count()
avg_ch_0 = df[df['is_children'] == 0]['debt'].sum() / df[df['is_children'] == 0]['debt'].count()
avg_ch_1 = df[df['is_children'] == 1]['debt'].sum() / df[df['is_children'] == 1]['debt'].count()
if avg_ch_0 <= avg_debt:
    print(f'Доля заемщиков без детей с задолженностью по кредиту не превышает среднее значение и составляет {avg_ch_0:.1%}')
else:
    print(f'Доля заемщиков без детей с задолженностью по кредиту больше среднего значения и составляет {avg_ch_0:.1%}')
if avg_ch_1 <= avg_debt:
    print(f'Доля заемщиков с детьми с задолженностью по кредиту не превышает среднее значение и составляет {avg_ch_1:.1%}')
else:
    print(f'Доля заемщиков с детьми с задолженностью по кредиту больше среднего значения и составляет {avg_ch_1:.1%}')

Доля заемщиков без детей с задолженностью по кредиту не превышает среднее значение и составляет 7.5%
Доля заемщиков с детьми с задолженностью по кредиту больше среднего значения и составляет 9.2%


**Вывод**

Доля заемщиков с детьми, имеющих задолженность по кредиту, превосходит как долю должников без детей, так и среднее значение по всей выборке. Предположение о том, что заемщики с детьми более ответственны не подтвердилось. Однако наличие детей у заемщика однозначно нужно учитывать при построении модели кредитного скоринга.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [17]:
#создание pivot таблицы для проверки гипотезы
data_pivot = df.pivot_table(index='family_status', values='debt', aggfunc='mean')
data_pivot

,debt
family_status,
в разводе,0.071130
вдовец / вдова,0.065693
гражданский брак,0.093471
женат / замужем,0.075452
не женат / не замужем,0.097509


**Вывод**

Доля должников среди заемщиков в различных категориях по семейному положения достаточно неоднородна. При этом сформулировать однозначный вывод довольно сложно- доля должников среди заемщиков в гражданском браке настолько же высокая, как и среди неженатых, а доли должников среди заемщиков в разводе и женатых близки и немного ниже среднего значения. Данный параметр можно учесть при построении модели кредитного скоринга, однако следует относиться к нему с осторожностью и постоянно отслеживать его изменение.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [18]:
#создание pivot таблицы для проверки гипотезы
data_pivot = df.pivot_table(index='total_income_type', values='debt', aggfunc='mean')
data_pivot

,debt
total_income_type,
100-й процентиль,0.069914
20-й процентиль,0.080168
40-й процентиль,0.083663
60-й процентиль,0.087879
80-й процентиль,0.084130


**Вывод**

Доля должников во всех выделенных категориях, кроме категории с наивысшим доходом, близки по значению, из чего можно сделать вывод, что уровень дохода не имеет сильного влияния на наличие задолженностей по кредитам у заемщиков с низким и средним доходом. Данный параметр не может использоваться в модели кредитного скоринга, так как не имеет ощутимого влияния.

- Как разные цели кредита влияют на его возврат в срок?

In [19]:
#создание pivot таблицы для проверки гипотезы
data_pivot = df.pivot_table(index='purpose', values='debt', aggfunc='mean')
data_pivot

,debt
purpose,
образование,0.092200
покупка автомобиля,0.093590
покупка недвижимости,0.072424
проведение свадьбы,0.080034
строительство и ремонт,0.072032


**Вывод**

Доля заемщиков, имеющих задолженность в категориях 'покупка недвижимости' и 'строительство и ремонт', ниже среднего значения и значений в остальных категориях. Заемщики в категориях 'образование' и 'покупка автомобиля' наоборот чаще имеют задолженности по кредитам. Данный параметр должен быть включен в модель кредитного скоринга.

* Как тип занятости влияет на возврат кредита в срок?

In [20]:
#создание pivot таблицы для проверки гипотезы
data_pivot = df.pivot_table(index='income_type', values='debt', aggfunc='mean')
data_pivot

,debt
income_type,
безработный,0.500000
в декрете,1.000000
госслужащий,0.059025
компаньон,0.074045
пенсионер,0.056412
предприниматель,0.000000
сотрудник,0.095724
студент,0.000000


**Вывод**

Как было сказано выше, выборка в категориях 'студент', 'предприниматель', 'в декрете' и 'безработный' статистически недостоверна, поэтому оценивать результаты в ней нет смысла. Однако для достоверных категорий видно, что пенсионеры и госслужащие намного реже допускают задолженности по кредитам, а сотрудники- намного выше среднего значения. Данный параметр должен быть учтен в модели кредитного скоринга.

* Как влияет уровень образования на возврат кредита в срок?

In [21]:
#создание pivot таблицы для проверки гипотезы
data_pivot = df.pivot_table(index='education', values='debt', aggfunc='mean')
data_pivot

,debt
education,
высшее,0.052952
начальное,0.109929
неоконченное высшее,0.091398
среднее,0.089902
ученая степень,0.000000


**Вывод**

Если не учитывать заемщиков с ученой степенью, так как данные по ним не могут быть признанными статистически достоверными, то можно сделать однозначный вывод о влиянии уровня образования на возврат кредита в срок- доля должников с высшим образованием более чем в 1,7 ниже, чем доля должников со средним образованием.

## Шаг 4. Общий вывод

При предварительной обработке данных выявлены следующие проблемы с сохранением данных:
* некорректное сохранение трудового стажа заемщиков- встречаются как отрицательные значения, так и слишком большие (более 1100 лет). Данная проблема, вероятнее всего, вызвана разными единицами измерения в разных источниках данных. Необходимо разработать единую методику внесения данных и ознакомить с ней работников банка.
* в столбце с возрастом заемщиков выявлено 101 нулевое значение. Необходимо ввести предварительную проверку значений в данном столбце при внесении данных.
* в таблице пропущены 2174 значения в столбцах с трудовым стажем и заработной платой в одних и тех же строках. Необходимо определить источник, из которого были получены данные, и исправить ошибку в данных.
* в форме внесения данных пункт 'Цель кредита' необходимо сделать выпадающим списком, а не текстовой формой. Такое решение позволит подробнее категоризировать данные и уточнить модель кредитного скоринга, так как в данный момент качество внесения данных в столбец зависит от добросовестности конкретного сотрудника банка.

По результатам исследования можно сделать вывод, что наибольшее влияние на вероятность возникновения задолженности по кредитам оказывают следующие факторы:
* наличие детей у заемщика;
* цель кредита;
* тип занятости заемщика;
* уровень образования заемщика.